This notebook explores a few different potential models of the Milky Way

In [ ]:
using CairoMakie
using Revise
using LilGuys, Arya

In [ ]:
# use natural units (G=1) for everything. if the potential uses different units
# we are best off to change the inputs rather than Agama's units
using PythonCall
agama = pyimport("agama")

In [ ]:
include("agama_utils.jl")

In [ ]:
obs_props_filename = ENV["DWARFS_ROOT"] * "/observations/sculptor/observed_properties.toml"

In [ ]:
# sculptor's adpoted mean orbit for this project
icrs = LilGuys.coord_from_file(obs_props_filename)
icrs_err = LilGuys.coord_err_from_file(obs_props_filename)

In [ ]:
gc = LilGuys.transform(Galactocentric, icrs)

In [ ]:
ep20 = load_agama_potential("EP2020.ini")


In [ ]:
orbit = calc_orbit(gc, ep20, units=:code, time=-10)

In [ ]:
plot_r_t(orbit)

In [ ]:
plot_y_z(orbit)

In [ ]:
mm11 = load_agama_potential("mcmillan11.ini")
mm17 = load_agama_potential("mcmillan17.ini")
bt08 = load_agama_potential("BT08.ini")
piffl14 = load_agama_potential("piffl_14.ini")
J95 = load_agama_potential("J95.ini")
galpy14 = load_agama_potential("galpy_2014.ini")
pouliasis17 = load_agama_potential("pouliasis+2017.ini")
ablimit20 = load_agama_potential("ablimit+2020.ini")
nitschai20 = load_agama_potential("nitschai+2020.ini")

In [ ]:
pouliasis17b = load_agama_potential("pouliasis+2017b.ini")
AS91 = load_agama_potential("AS1991.ini")

In [ ]:
potentials_light = [
    "EP20" => ep20,
    "bt08" => bt08,
    "galpy14" => galpy14,
    "pouliasis+17" => pouliasis17,
    "AS1991" => AS91,
    ]


In [ ]:
potentials_heavy = [
    "EP20" => ep20,
    "mcmillan 11" => mm11,
    "piffl14" => piffl14,
    "J95" => J95,
    "pouliasis+17b" => pouliasis17,


]

In [ ]:
potentials = [
    "EP20" => ep20,
    "mcmillan 17" => mm17,
    "ablimit + 2020" => ablimit20,
    "nitschai+20" => nitschai20,

]

# Potential comparisons

In [ ]:
function compare_v_circ(potentials)
    fig = Figure()
    ax = Axis(fig[1, 1])

    for (label, pot) in potentials
        plot_v_circ!(pot, label=label)
    end

    axislegend(position=:rb)
    fig
end

In [ ]:
function plot_all(gc, pot)
    orbit = calc_orbit(gc, pot)
    plot_r_t(orbit) |> display
    plot_y_z(orbit) |> display
end

In [ ]:
function plot_all(gc, pots)

    
    orbits = [calc_orbit(gc, pot, units=:code, time=10 / T2GYR) for (name, pot) in pots]

    display(compare_v_circ(pots))
    
    fig = Figure()
    ax = axis_r_t(fig[1, 1])
    
    for i in 1:length(pots)
        plot_r_t!(ax, orbits[i], label=pots[i].first)
    end

    axislegend()
    
    display(fig)


    fig = Figure()
    ax = axis_y_z(fig[1,1])


    for i in 1:length(pots)
        plot_y_z!(ax, orbits[i], label=pots[i].first)
    end

    axislegend()
    display(fig)
end

In [ ]:
plot_all(gc, potentials)

In [ ]:
plot_all(gc, potentials_light)

In [ ]:
plot_all(gc, potentials_heavy)

# Old Cord

In [ ]:
R200_NFW(M200) / c

In [ ]:
M200 = 66.3*M0
c = 12.36
Ms = Ms_NFW( M200, c)

Rs = 14.45*u.kpc
Ms

In [ ]:
find_M_c(Ms, Rs)

In [ ]:
M200 = 6.749*M0
c = 12.7988
Ms = Ms_NFW(M200, c)

In [ ]:
Ms

In [ ]:
ablimit_c = {
    "halo": NFW(amp=Ms, a=Rs),
    "disk": pouliasis17_thick + pouliasis17_thin,
    "bulge": pouliasis17_bulge,
}

ablimit20a = [val for key, val in ablimit_c.items()]




In [ ]:
M200 = 82.2*M0
c = 13.04
Ms = Ms_NFW(M200, c)
ablimit_b_comp = {
    "halo": NFW(amp=Ms, a=14.71*u.kpc),
    "disk": pouliasis17_thick + pouliasis17_thin,
    "bulge": pouliasis17_bulge,
}

ablimit20b = [val for key, val in ablimit_b_comp.items()]


In [ ]:
R200_NFW(M200) / c

In [ ]:
shen22_halo = gp.PowerSphericalPotential(amp=0.2275*M0, r1=1*u.kpc, alpha=2+0.43)


c=12
R200=207*u.kpc
Ms = Ms_NFW(105*M0, c) # is 95 in paper but not described well...
shen22_halo_b = NFW(amp=Ms, a=R200/c)

In [ ]:
shen22_halo.mass(100*u.kpc) / M0

In [ ]:
shen22_halo.mass(250*u.kpc) / M0

In [ ]:
Rs = np.linspace(10, 250, 1000) * u.kpc

In [ ]:
plt.plot(Rs, [shen22_halo.mass(R) / M0 for R in Rs])
plt.plot(Rs, [shen22_halo_b.mass(R) / M0 for R in Rs])

In [ ]:
plot_Vc(shen22_halo, R_min=10)
plot_Vc(galpy14)
plot_Vc(shen22_halo_b, R_min=10)

In [ ]:
nitschai_halo = gp.TwoPowerTriaxialPotential(amp=41.1*M0, alpha=1.53, beta=3, a=16.8*u.kpc, c=1.14)
nitschai_halo_b = gp.TwoPowerSphericalPotential(amp=44.1*M0, alpha=1.53, beta=3, a=16.8*u.kpc)

In [ ]:
Mh = nitschai_halo.mass(8.2*u.kpc)

In [ ]:
nitschai_halo.dens(8.2*u.kpc, 0*u.kpc)

In [ ]:
Mh / (Mh + mcmillan2017_halo.mass(8.2*u.kpc) + mcmillan2017_thick.mass(8.2*u.kpc) +  mcmillan2017_thin.mass(8.2*u.kpc))

In [ ]:
nitschai = nitschai_halo + mcmillan2017_bulge + mcmillan2017_thick + mcmillan2017_thick + mcmillan2017_h1 + mcmillan2017_h2
nitschai_b = nitschai_halo_b + mcmillan2017_bulge + mcmillan2017_thick + mcmillan2017_thick + mcmillan2017_h1 + mcmillan2017_h2

In [ ]:
A_disk = 0.056 / 0.2

In [ ]:
Ms = 105*M0
r_s = 20.2*u.kpc
heavy_halo = NFW(amp=Ms, a=r_s)
find_M_c(Ms, r_s)

In [ ]:
Ms = 60*M0
r_s = 20.2*u.kpc
light_halo = NFW(amp=Ms, a=r_s)
find_M_c(Ms, r_s)

In [ ]:
Ms = 130*M0
r_s = 33*u.kpc
me_flat_halo =  NFW(amp=Ms, a=r_s)
me_flat = EP2020_thin + EP2020_thick + EP2020_bulge + me_flat_halo

find_M_c(Ms, r_s)

In [ ]:
Ms = 45*M0
r_s = 10*u.kpc
me_steep_halo =  NFW(amp=Ms, a=r_s)

me_steep = EP2020_thin + EP2020_thick + EP2020_bulge + me_steep_halo

find_M_c(Ms, r_s)

In [ ]:
heavy_mw = EP2020_bulge + EP2020_thin + EP2020_thick + heavy_halo
light_mw = EP2020_bulge + EP2020_thin + EP2020_thick + light_halo

In [ ]:
potentials = {
    "EP20": EP2020,
    #"mcmillan11": mcmillan2011, # mcmillan is very close to EP2020
    #"mcmillan17": mcmillan2017,
    #"mcmillan17b": mcmillan2017_nogas,
    "galpy14": galpy14,
    # "galpy14_heavy": galpy14_heavy,
    # "pouliaris17": pouliasis17,
    # #"pouliaris17b": pouliasis17b,
    # "ablimit": ablimit20a,
    # "flat": me_flat,

}